## Extraer features de los modelos

In [ ]:
import numpy as np
import librosa
import warnings
import pandas as pd
import os

# Ignorar advertencias específicas de librosa
warnings.filterwarnings('ignore', category=UserWarning)

def extract_important_features(file_path, offset=None, duration=None):
    # Cargar el archivo de audio
    y, sr = librosa.load(file_path, sr=None, offset=offset, duration=duration)
    S = librosa.stft(y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_stft_mean = np.mean(chroma_stft)
    chroma_stft_var = np.var(chroma_stft)
    rms = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms)
    rms_var = np.var(rms)
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_centroid_mean = np.mean(spec_centroid)
    spectral_centroid_var = np.var(spec_centroid)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_bandwidth_mean = np.mean(spec_bw)
    spectral_bandwidth_var = np.var(spec_bw)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = np.mean(rolloff)
    rolloff_var = np.var(rolloff)
    zcr = librosa.feature.zero_crossing_rate(y)
    zero_crossing_rate_mean = np.mean(zcr)
    zero_crossing_rate_var = np.var(zcr)
    harmony = librosa.effects.harmonic(y)
    harmony_mean = np.mean(harmony)
    harmony_var = np.var(harmony)
    perceptr = librosa.feature.melspectrogram(y=y, sr=sr)
    perceptr_mean = np.mean(librosa.power_to_db(perceptr, ref=np.max))
    perceptr_var = np.var(librosa.power_to_db(perceptr, ref=np.max))
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    mfcc_means = [np.mean(m) for m in mfccs]
    mfcc_vars = [np.var(m) for m in mfccs]


    features = {
        'song_title': os.path.basename(file_path),
        'chroma_stft_mean': chroma_stft_mean,
        'chroma_stft_var': chroma_stft_var,
        'rms_mean': rms_mean,
        'rms_var': rms_var,
        'spectral_centroid_mean': spectral_centroid_mean,
        'spectral_centroid_var': spectral_centroid_var,
        'spectral_bandwidth_mean': spectral_bandwidth_mean,
        'spectral_bandwidth_var': spectral_bandwidth_var,
        'rolloff_mean': rolloff_mean,
        'rolloff_var': rolloff_var,
        'zero_crossing_rate_mean': zero_crossing_rate_mean,
        'zero_crossing_rate_var': zero_crossing_rate_var,
        'harmony_mean': harmony_mean,
        'harmony_var': harmony_var,
        'perceptr_mean': perceptr_mean,
        'perceptr_var': perceptr_var,
        'tempo': tempo,
    }
    # Agregar MFCCs
    for i in range(20):
        features[f'mfcc{i+1}_mean'] = mfcc_means[i]
        features[f'mfcc{i+1}_var'] = mfcc_vars[i]

    # Guardar las features para comprobar los valores
    # df = pd.DataFrame([features])
    # df.to_csv('features.csv', index=False)

    return features

## Generar spectograma de cada canción

In [ ]:
import matplotlib.pyplot as plt
import librosa.display

def save_spectrogram(file_path, genre, output_base="../data/spectrograms"):
    try:
        y, sr = librosa.load(file_path, sr=None)
        S = librosa.feature.melspectrogram(y=y, sr=sr)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # Crear la carpeta si no existe
        output_dir = os.path.join(output_base, genre)
        os.makedirs(output_dir, exist_ok=True)

        # Nombre del archivo de salida
        file_name = os.path.splitext(os.path.basename(file_path))[0] + ".png"
        output_path = os.path.join(output_dir, file_name)

        # Guardar espectrograma
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
        plt.axis('off')  # sin ejes
        plt.tight_layout()
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    except Exception as e:
        print(f"Error al generar espectrograma de {file_path}: {e}")


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

BASE_DIR = "../data/music"

features_30_sec = []
features_3_sec = []

# Se recorre cada género
for genre in os.listdir(BASE_DIR):
    genre_path = os.path.join(BASE_DIR, genre)
    if not os.path.isdir(genre_path):
        continue

    print(f"Procesando género: {genre}")
    for file in tqdm(os.listdir(genre_path)):
        if not file.endswith(".wav"):
            continue

        file_path = os.path.join(genre_path, file)

        # --- Features para canción completa ---
        try:
            f_full = extract_important_features(file_path)
            f_full["genre"] = genre
            features_30_sec.append(f_full)
            save_spectrogram(file_path, genre)
            
        except Exception as e:
            print(f"Error en {file_path} (30s):", e)

        # --- Features para segmentos de 3s ---
        for i in range(10):
            try:
                f_seg = extract_important_features(file_path, offset=i*3, duration=3)
                f_seg["genre"] = genre
                f_seg["segment"] = i  # opcional
                features_3_sec.append(f_seg)
            except Exception as e:
                print(f"Error en {file_path} (3s offset {i*3}):", e)

# Guardamos los CSV
df_full = pd.DataFrame(features_30_sec)
df_full.to_csv("../data/raw/actual_features_30_sec.csv", index=False)

df_segments = pd.DataFrame(features_3_sec)
df_segments.to_csv("../data/raw/actual_features_3_sec.csv", index=False)

print("Archivos CSV y espectrogramas guardados correctamente.")


Procesando género: blues


100%|██████████| 100/100 [06:59<00:00,  4.19s/it]


Procesando género: classical


100%|██████████| 100/100 [09:26<00:00,  5.67s/it]


Procesando género: country


...

Procesando género: reggae


100%|██████████| 100/100 [07:00<00:00,  4.20s/it]


Procesando género: rock


100%|██████████| 100/100 [07:01<00:00,  4.22s/it]


¡Listo! Se guardaron los CSV.


Se extrayeron hasta 60 features del dataset de canciones de 30 segundos, las cuáles están clasificadas en 10 géneros:
- **blues**
- **classical**
- **country**
- **disco**
- **hiphop**
- **jazz**
- **metal**
- **pop**
- **reggae**
- **rock**

Se generaron los siguientes archivos:

- **[actual_features_30_sec.csv](actual_features_30_sec.csv)**:
    - Contienen un total de 59 columnas y 100 filas de cada género.
- **[actual_features_3_sec.csv](actual_features_3_sec.csv)**:
    - Contienen un total de 60 columnas y 1000 filas de cada género.
    - Se agregó una columna `segment` que indica la parte de la canción a la que corresponde cada fila.
    - Para obtener más filas, se dividieron las canciones en 10 partes de 3 segundos cada una.
- **[spectrograms/\*/\*.png](spectrograms/actual_features_30_sec.csv)**
    - Organizado en carpetas por género, se encuentran los espectrogramas de las 1000 canciones originales.